In [2]:
#Skripte laden
%reload_ext autoreload
%autoreload 2
import os
os.chdir('/home/safiul/email-classification')
import data
from Email_Classification.Functions.packages import *
import Email_Classification.Functions.Functions as fn
import Email_Classification.Functions.parameters as par

In [52]:
#Pfade einsetzen, Anzahl der Features und ModellKategorie festlegen
num_feature = 2000
model_name = 'NB'
result = pd.DataFrame(columns = ['Model', 'N-Frature', 
                                 'Accuracy', 'Pred. Prob', 'Entropy','lr'])
path_model = "Email_Classification/Models/Nonsequential Models/Naive Bayes/tfidf"
path_feature = 'Email_Classification/Models/Nonsequential Models/feature/tfidf'
X_test_path = os.path.join(path_feature, 'feature_{}/X_test.npz'.format(num_feature))
y_test_path = 'Email_Classification/Models/Nonsequential Models/feature/tfidf/label/y_test.npy'
result_path = os.path.join(path_model, 'result/result.csv')

In [21]:
#Testmenge und Modell laden
model = fn.readmodel(path_model, makeZip = False)
X_test_tfidf = scipy.sparse.load_npz(X_test_path)
with open(y_test_path, 'rb') as f:
    y_test = np.load(f)
encoder = LabelBinarizer()
label_binary = encoder.fit_transform(y_test)
mod = model['{}_{}'.format(model_name, num_feature)]
if model_name == 'LGB':
    pred_prob = mod.predict(X_test_tfidf)
else:
    pred_prob = mod.predict_proba(X_test_tfidf)

In [ ]:
%%time
#Erstellt Ergebnistabelle
Accuracy, auroc_final , auroc_metaclf,_,_,_ = \
fn.result_final(pred_prob = pred_prob, y_test = label_binary)
res = np.zeros(6)
res = np.append([model_name], num_feature)
res = np.append(res, Accuracy)
res = np.append(res, auroc_final)
res = np.append(res, auroc_metaclf)
res_series = pd.Series(res, index = result.columns)
result = result.append(res_series, ignore_index=True)
result.iloc[:, 2:6] = result.iloc[:, 2:6].apply(pd.to_numeric).round(4)
result.to_csv(path_or_buf =  result_path, index = False)

In [ ]:
#Ein Wörterbuch für Unsicherheitsmetriken erstellen
#df_final = dict()
dict_meta = { 'pred. prob.': feature_test_meta[:,0],
            'Entropy': -feature_test_meta[:,1],
            'lr': lr.predict_proba(feature_test_meta)[:,1],
            'labels': label_test_meta}
df_meta = pd.DataFrame(dict_meta)
df_meta['labels'] = df_meta['labels'].apply(lambda x: 'korrekt (NB)' if x == 1 else 'falsch (NB)')
df_final['NB'] = df_meta
#with open('Email_Classification/final_scripts/result/data_plot/ML/df_final.pkl', 'rb') as f:
#    pickle.dump(df_final, f)

In [58]:
#Ergebnisse in einer tex Datei speichern
result = pd.read_csv(result_path)
result = result.drop(['Model'], axis = 1)
for i in ['Accuracy', 'Pred. Prob', 'Entropy','lr']:
    result[i] = result[i].map(lambda n: '{:,.2%}'.format(n))
result.columns = ['Anzahl der Features', 'Genauigkeit', 'pkw', 'Entropie', 'lr']
result.to_latex(os.path.join(path_model, 'result/tex/result_{}.tex'.format(model_name)), index = False)

In [ ]:
#Violin-Plot für die Unsicherheitsverteilungen
with open('Email_Classification/final_scripts/result/data_plot/ML/df_final.pkl', 'rb') as f:
    df_final = pickle.load(f)
#matplotlib.rcParams.update({'font.size': 50})
sns.set(font_scale= 2.5)
sns.set_style("white")

plt.figure(figsize=(40,25))
plt.subplot(3, 4, 1)
fig1 = sns.violinplot(x = df_final['NB']['labels'],  
              y =  df_final['NB']['pred. prob.'] ,
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('WpK')
#plt.setp(fig1.get_yticklabels(), rotation=90)

plt.subplot(3, 4, 2)
fig1 = sns.violinplot(x = df_final['LR']['labels'],  
              y =  df_final['LR']['pred. prob.'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('WpK')
#plt.setp(fig1.get_yticklabels(), rotation=45)

plt.subplot(3, 4, 3)
fig1 = sns.violinplot(x = df_final['RF']['labels'],  
              y =  df_final['RF']['pred. prob.'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('WpK')
#plt.setp(fig1.get_yticklabels(), rotation=90)


plt.subplot(3, 4, 4)
fig1 = sns.violinplot(x = df_final['LGBM']['labels'],  
              y =  df_final['LGBM']['pred. prob.'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('WpK')
#plt.setp(fig1.get_yticklabels(), rotation=90)



plt.subplot(3, 4, 5)
fig2 = sns.violinplot(x = df_final['NB']['labels'],  
              y = df_final['NB']['Entropy'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('Entropie')
#plt.setp(fig2.get_yticklabels(), rotation=90)


plt.subplot(3, 4, 6)
fig2 = sns.violinplot(x = df_final['LR']['labels'],  
              y = df_final['LR']['Entropy'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('Entropie')
#plt.setp(fig2.get_yticklabels(), rotation=90)

plt.subplot(3, 4, 7)
fig2 = sns.violinplot(x = df_final['RF']['labels'],  
              y = df_final['RF']['Entropy'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('Entropie')
#plt.setp(fig2.get_yticklabels(), rotation=90)

plt.subplot(3, 4, 8)
fig2 = sns.violinplot(x = df_final['LGBM']['labels'],  
              y = df_final['LGBM']['Entropy'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('Entropie')
#plt.setp(fig2.get_yticklabels(), rotation=90)


plt.subplot(3, 4, 9)
fig3 = sns.violinplot(x = df_final['NB']['labels'],  
              y =  df_final['NB']['lr'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('LR (Metaklassif.)')
#plt.setp(fig3.get_yticklabels(), rotation=90)

plt.subplot(3, 4, 10)
fig3 = sns.violinplot(x = df_final['LR']['labels'],  
              y =  df_final['LR']['lr'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('LR (Metaklassif.)')
#plt.setp(fig3.get_yticklabels(), rotation=90)

plt.subplot(3, 4, 11)
fig3 = sns.violinplot(x = df_final['RF']['labels'],  
              y =  df_final['RF']['lr'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('LR (Metaklassif.)')
#plt.setp(fig3.get_yticklabels(), rotation=90)

plt.subplot(3, 4, 12)
fig3 = sns.violinplot(x = df_final['LGBM']['labels'],  
              y =  df_final['LGBM']['lr'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('LR (Metaklassif.)')
#plt.setp(fig3.get_yticklabels(), rotation=90)

plt.savefig('Email_Classification/Models/Nonsequential Models/\
result/figure/dist_meta_classif.pdf', bbox_inches = 'tight',
    pad_inches = 0.1)
plt.show()

In [74]:
prob_kl['prob_lr_df_RF']['labels'] = prob_kl['prob_lr_df_RF']['labels'].\
apply(lambda x: 'right (RF)' if x == 'right' else 'wrong (RF)')

In [ ]:
#Gegenüberstellung der Genauigkeit und Unsicherheit
row_result = pd.read_csv('Email_Classification/Models/Nonsequential Models/result/result.csv')
sns.set(font_scale= 1.9)
sns.set_style("white")
matplotlib.rcParams.update({'font.size': 20})
plt.rc('axes', labelsize=25) 
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(27, 10))
groups = row_result.groupby('Model')
markers =  ['*', 'o', 'v', '^']
markerfacecolor = ['blue', 'green', 'red',  'magenta']
s = 20
i = 0
for name, group in groups:
    mi = markers[i]
    mf = markerfacecolor[i]
    i = i +1
    x = group['Accuracy']
    y = group['AUROC/Pred. Prob']
    ax1.plot(x,y, marker = mi, markerfacecolor = mf, linestyle = "", label = name, markersize= s)
    m, b = np.polyfit(row_result['Accuracy'], row_result['AUROC/Pred. Prob'], 1)
    ax1.plot(row_result['Accuracy'], m*row_result['Accuracy'] + b, color = 'black')
    ax1.legend(loc = 'lower right', fontsize = 'large')
    ax1.set(xlabel = 'Genauigkeit', ylabel = 'AUROC (WpK)')


i = 0
for name, group in groups:
    mi = markers[i]
    mf = markerfacecolor[i]
    i = i +1
    x = group['Accuracy']
    y = group['AUROC/Entropy']
    ax2.plot(x,y, marker = mi, markerfacecolor = mf, linestyle = "", label = name, markersize= s)
    m, b = np.polyfit(row_result['Accuracy'], row_result['AUROC/Entropy'], 1)
    ax2.plot(row_result['Accuracy'], m*row_result['Accuracy'] + b, color = 'black')
    ax2.legend(loc = 'lower right', fontsize = 'large')
    ax2.set(xlabel = 'Genauigkeit', ylabel = 'AUROC (Entropie)')

i = 0
for name, group in groups:
    mi = markers[i]
    mf = markerfacecolor[i]
    i = i +1
    x = group['Accuracy']
    y = group['AUROC/lr']
    ax3.plot(x,y, marker = mi, markerfacecolor = mf, linestyle = "", label = name, markersize= s)
    m, b = np.polyfit(row_result['Accuracy'], row_result['AUROC/lr'], 1)
    ax3.plot(row_result['Accuracy'], m*row_result['Accuracy'] + b, color = 'black')
    ax3.legend(loc = 'lower right', fontsize = 'large')
    ax3.set(xlabel = 'Genauigkeit', ylabel = 'AUROC (LR-Metaklassif)')
#fig.text(0.5, 0.02, 'accuracy', ha='center', fontsize=12)
#fig.text(0.06, 0.5, 'AUROC', va='center', rotation='vertical', fontsize=12)
plt.savefig('Email_Classification/Models/Nonsequential Models/\
result/figure/auroc_vs_acc.pdf', bbox_inches = 'tight',
    pad_inches = 0.1)
plt.show()  

In [11]:
#Erstellung einer Tabelle mit der Ergebnisse des Kolmogorov-Smrinoff-Tests
model = 'LGBM'
df1 = df_final[model][df_final[model].labels == 'korrekt ({})'.format(model)]
df2 = df_final[model][df_final[model].labels != 'korrekt ({})'.format(model)]
dict_test = { 'pred. prob.': [scipy.stats.ks_2samp(df1['pred. prob.'], df2['pred. prob.']).statistic,
                              scipy.stats.ks_2samp(df1['pred. prob.'], df2['pred. prob.']).pvalue],
            'Entropy': [scipy.stats.ks_2samp(df1['Entropy'], df2['Entropy']).statistic,
                              scipy.stats.ks_2samp(df1['Entropy'], df2['Entropy']).pvalue],
            'lr': [scipy.stats.ks_2samp(df1['lr'], df2['lr']).statistic,
                              scipy.stats.ks_2samp(df1['lr'], df2['lr']).pvalue]
            }
df_test = pd.DataFrame(dict_test)
test_final = test_final.append(df_test)
test_final = test_final.round(3)
test_final.to_latex('Email_Classification/Models/Sequential Models/result/result_test.tex')

In [ ]:
#Kritischer Wert für KS-Test berechnen
criticalvalue_KS(alpha = 0.01, n = df1, m = df2).round(4)